### Load packages etc, and load the child-class of gmspython, "Abatement", and load "techdata_to_tree" which converts a technology catalog into production trees

In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
os.chdir(py['main'])
import abatement, techdata_to_tree, sys, ShockFunction
os.chdir(curr)
data_folder = os.getcwd()+'\\Data'
gams_folder = data_folder + "\\..\\gamsmodels\\Main"

# **1.1 Construct trees**

#### Load technology data.
This runs the script in techdata_to_tree:

In [2]:
# inputfile = "techdata_newID.xlsx"
inputfile = "techdata_newID.xlsx"
#inputfile = "techdata_ID_oil.xlsx"
output = techdata_to_tree.load_techcats(pd.read_excel(data_folder + "/" + inputfile, sheet_name=["inputdisp", "endofpipe", "inputprices"]))

#### The output of the code is a dictionary with three upper keys:

In [3]:
output.keys()

dict_keys(['ID', 'inputprices', 'EOP'])

The two different modules correspond to the two types of technology catalogs that the model can handle.\
*Input-displacing* (ID) and *End of pipe* (EOP).

In [4]:
modules = ["ID", "EOP"]

ID contains a dictionary related to input-displacing technologies, EOP does so for end of pipe (we use this naming convention throughout).\
They contain the following keys:

In [5]:
for module in modules:
    print("Keys of " + module + ": ", output[module].keys(), "\n")

Keys of ID:  dict_keys(['techs_inputs', 'techs', 'components', 'upper_categories', 'mu', 'Q2P', 'unit_costs', 'current_coverages', 'coverage_potentials', 'IO_tech', 'IO_tech_inputs', 'baseline_U_inputs']) 

Keys of EOP:  dict_keys(['techs_inputs', 'techs', 'components', 'upper_categories', 'mu', 'Q2P', 'unit_costs', 'current_coverages', 'coverage_potentials']) 



As an example, "techs" shows, for each technology, which technology good it produces, e.g. for input-displacing:

In [6]:
output["ID"]["techs"]

{'ID_1': ['U_ID_1_1', 'U_ID_1_2', 'U_ID_1_3'],
 'ID_2': ['U_ID_2_1', 'U_ID_2_2', 'U_ID_2_3'],
 'ID_3': ['U_ID_3_1', 'U_ID_3_2'],
 'ID_4': ['U_ID_4_1', 'U_ID_4_2', 'U_ID_4_3'],
 'ID_5': ['U_ID_5_1', 'U_ID_5_2']}

And "upper_categories" shows the mapping between energy services (E) and their respective components (C) for ID.\
All of the trees in the `output` object are dictionaries where the keys are nodes in the tree and the values are lists of connected branches.

In [7]:
output["ID"]["upper_categories"]

{'EL': ['C_EL_1', 'C_EL_2', 'C_EL_3', 'C_EL_4', 'C_EL_5', 'C_EL_base'],
 'EH': ['C_EH_1', 'C_EH_2', 'C_EH_3', 'C_EH_base'],
 'ER': ['C_ER_1', 'C_ER_2', 'C_ER_base']}

... and the mapping between emission types M ($CO_2$, $SO_2$ etc.) and the components C.\
Note importantly that this mapping does not constitute an actual part of the tree, because components are the outputs of the EOP sector.\

In [8]:
output["EOP"]["upper_categories"]

{'CO2': [], 'SO2': ['C_SO2_1'], 'NOX': []}

### Initialize nesting tree, call it "Abatement"
The construction of the production tree starts with an initialization of the *nesting_tree* class:

In [9]:
nt = nesting_tree.nesting_tree(name='Abatement')

The "trees" attribute starts off empty, reflecting that no (sub)trees have been added yet. 

In [10]:
nt.trees

{}

#### The following cells add each of the subtrees to the tree-object, "nt"

##### First, input-displacing subtrees

Upper part, connecting energy services to components (this is an input-tree, so we do not supply additional keyword arguments).\
Again, we use the naming convention that a prefix reflects whether the tree is related to input-displacement (ID) or end of pipe (EOP).
The prefix is followed by an underscore and then letters reflecting which elements in the tree are connected.

The first tree connects energy services (E) to components (C), related to input-displacing technologies (ID)

In [11]:
nt.add_tree(output["ID"]["upper_categories"], tree_name = 'ID_EC', **{"type_f":"CES_norm"})

Before proceeding to the next tree, lets print some information from this tree.\
First, we see that the tree has been added to the list of trees in the "aggregate" tree, nt:

In [12]:
nt.trees

{'ID_EC': <nesting_trees.nt at 0x1fdc4b4f5c8>}

Second, information about the tree is automatically added when the tree is added, e.g. the "type_f" attribute states that the functional form in this subtree is constant elasticity of substitution (CES)

In [13]:
nt.trees["ID_EC"].__dict__

{'name': 'ID_EC',
 'tree': {'EL': ['C_EL_1',
   'C_EL_2',
   'C_EL_3',
   'C_EL_4',
   'C_EL_5',
   'C_EL_base'],
  'EH': ['C_EH_1', 'C_EH_2', 'C_EH_3', 'C_EH_base'],
  'ER': ['C_ER_1', 'C_ER_2', 'C_ER_base']},
 'type_io': 'input',
 'version': 'std',
 'n': 'n',
 'nn': 'nn',
 'nnn': 'nnn',
 'map_': 'map_ID_EC',
 'kno': 'kno_ID_EC',
 'bra': 'bra_ID_EC',
 'inp': 'inp_ID_EC',
 'out': 'out_ID_EC',
 'temp_namespace': None,
 'type_f': 'CES_norm',
 'database': <DataBase.GPM_database at 0x1fdc4b4fc48>}

However, even though the namespace includes e.g. "map_":"map_ID_EC", the database of the tree still does not contain this mapping. This requires running the run() method.\
Instead of doing that for each tree individually, we instead add all trees and call the run_all() methods which also calls the run() on each tree.

Next, we add the middle part, connecting components C to their technology goods U

In [14]:
nt.add_tree(output["ID"]["components"], tree_name = "ID_CU", **{"type_f":"MNL"})

The two remaining parts are the bottom ones. \
First, the one that connects technologies to their outputs (technology goods). That these are outputs is specified explicitly by using the `type_io` keyword.\
Second, the one that connects technologies to their inputs (non-capital inputs X, and capital K)

In [15]:
nt.add_tree(output["ID"]["techs"], tree_name="ID_TU", **{'type_io': 'output', 'type_f': 'CET_norm'})
nt.add_tree(output["ID"]["techs_inputs"], tree_name="ID_TX")

Baseline components and technology goods, and their respective sets of inputs are also part of the aggregate tree. We distinguish between those with their own inputs (these baseline technologies are the ones that come from knowing which energy mix a (set of) technologies replaces. The remaining baseline technology goods (and all baseline components by construction) are outputs of the "IO technology". This IO technology in turn draws on all inputs from the economy. This is the one we calibrate to make sure we replicate IO data. 

In [16]:
nt.add_tree(output["ID"]["IO_tech"], tree_name="ID_IOCU", **{"type_io":"output", "type_f":"CET_norm"})
nt.add_tree(output["ID"]["IO_tech_inputs"], tree_name="ID_IOX")
nt.add_tree(output["ID"]["baseline_U_inputs"], tree_name="ID_UbaseX")

#### Next, we add the end of pipe subtrees.
First, the one connecting components (which are the final product in the end-of-pipe tree) to the technology goods from which they are created:

In [17]:
nt.add_tree(output["EOP"]["components"], tree_name = "EOP_CU")

Second, the bottom part. This consists of technologies and their outputs and inputs respectively, similarly to with input-displacing technologies:

In [18]:
nt.add_tree(output["EOP"]["techs"], tree_name="EOP_TU", **{'type_io': 'output', 'type_f': 'CET_norm'})
nt.add_tree(output["EOP"]["techs_inputs"], tree_name="EOP_TX")

##### Now, all trees have been added:

In [19]:
nt.trees

{'ID_EC': <nesting_trees.nt at 0x1fdc4b4f5c8>,
 'ID_CU': <nesting_trees.nt at 0x1fdc4bcec08>,
 'ID_TU': <nesting_trees.nt at 0x1fdc423d348>,
 'ID_TX': <nesting_trees.nt at 0x1fdc1148d48>,
 'ID_IOCU': <nesting_trees.nt at 0x1fdc4bb5048>,
 'ID_IOX': <nesting_trees.nt at 0x1fdc4bb5388>,
 'ID_UbaseX': <nesting_trees.nt at 0x1fdc4b4fa08>,
 'EOP_CU': <nesting_trees.nt at 0x1fdc4bd1988>,
 'EOP_TU': <nesting_trees.nt at 0x1fdc4adf888>,
 'EOP_TX': <nesting_trees.nt at 0x1fdc11162c8>}


The next step is to use the method `run_all`. This runs through a number of steps, where it sets up sets/subsets/mappings identifying which elements e.g. are inputs, intermediate goods, and outputs in the aggregate tree, i.e. the combination of the subtrees we just added. *Tutorial_nesting_tree* includes a brief review of these.\
There are still some of the objects in `output` that we have not used. We will return to these later as they become relevant (the Q2P part will be ignored altogether though).

In [20]:
nt.run_all()

This constructs the aggregate tree. \
For a few highlights, let's check:
1. The outputs of the aggregate tree (aggregate simply refers to the combination of all the individual (sub)trees\
This includes energy services (from ID) and components (from EOP)

In [21]:
list(nt.database.series["out"].vals)

['EL', 'EH', 'ER', 'C_SO2_1']

2. The outputs of a particular tree, say EOP_TU\
An error here? It says Ts are the outputs from an output tree with Ts as nods and Us as branches.\
I guess you could call it an "error", but it does not matter, because: The aggregate tree is constructed correctly and\
the variable showing outputs "out_\[treename\]" is a temporary variable, later to be discarded (see the key prune_trees).

In [22]:
list(nt.trees["EOP_TU"].database.series["out_EOP_TU"].vals)

[]

3. Likewise, we can check th inputs of EOP_TU\
Likewise, this reports that the Us are inputs, which they really are not.

In [23]:
nt.trees["EOP_TU"].database.series["inp_EOP_TU"].vals

Index(['U_EOP_t1_1'], dtype='object', name='n')

4. The outputs of EOP_TU that are also outputs in the aggregate tree\
Correctly says that those Ts are not outputs of the aggregate tree (which were identified correctly in point 1 above)

In [24]:
list(nt.trees["EOP_TU"].database.series["out_EOP_TU"].vals)

[]

Lastly, lets print the objects that the `nt` class instance contains:

In [25]:
nt.__dict__

{'name': 'Abatement',
 'version': 'std',
 'trees': {'ID_EC': <nesting_trees.nt at 0x1fdc4b4f5c8>,
  'ID_CU': <nesting_trees.nt at 0x1fdc4bcec08>,
  'ID_TU': <nesting_trees.nt at 0x1fdc423d348>,
  'ID_TX': <nesting_trees.nt at 0x1fdc1148d48>,
  'ID_IOCU': <nesting_trees.nt at 0x1fdc4bb5048>,
  'ID_IOX': <nesting_trees.nt at 0x1fdc4bb5388>,
  'ID_UbaseX': <nesting_trees.nt at 0x1fdc4b4fa08>,
  'EOP_CU': <nesting_trees.nt at 0x1fdc4bd1988>,
  'EOP_TU': <nesting_trees.nt at 0x1fdc4adf888>,
  'EOP_TX': <nesting_trees.nt at 0x1fdc11162c8>},
 'database': <DataBase.GPM_database at 0x1fdc4678b08>,
 'n': 'n',
 'nn': 'nn',
 'nnn': 'nnn',
 'inp': 'inp',
 'out': 'out',
 'int': 'int',
 'fg': 'fg',
 'wT': 'wT',
 'map_all': 'map_all',
 'kno_out': 'kno_out',
 'kno_inp': 'kno_inp',
 'prune_trees': {'OnlyQ', 'bra', 'inp', 'kno', 'out'}}

# **1.2 Adding parameters to the database**

The next step is to construct a database that contains all the share-parameters that we can deduce from technology data, as well as appropriate starting values for endogenous variables. These starting values will be set to the value that they would have if the entire tree was Leontief.
The end goal is to make sure the database contained by `nt` includes all these parameters and starting values. 
To do so, we construct an empty database, add the share-parameters and starting values to this, and then finally merge that database with the one in `nt`.

The GPM database of the nesting_tree instance (`nt`) does not contain the $\mu$ parameters calculated from technology data yet.\
The the $\mu$-parameters are stored under the key of the same name in `output`.\
Let's check out the database of `nt` now, to see that it does not include any symbol called $\mu$ yet:

In [26]:
print("The database nt.database is of type " + str(type(nt.database)))
print("\nDoes the database include a symbol called mu? ") 
if "mu" in nt.database.series:
    print("YES!")
else:
    print("NO!")
#print("The database contains the following:")
#nt.database.series.__dict__

The database nt.database is of type <class 'DataBase.GPM_database'>

Does the database include a symbol called mu? 
NO!


First we create an empty database:

In [27]:
db = DataBase.GPM_database()

We store the share-parameters from technology data in an object called `mu`. This does not contain all share-parameters of the tree (e.g. it does not include the share parameters of the inputs under baseline technology goods, or the share parameters for technology goods under components).

In [28]:
mu = output["ID"]["mu"].append(output["EOP"]["mu"])

Check out the mu parameters (prints just the five first entries here).\
Note: The mu-object is a series with a multiindex, where the first level is called `n` and the second level `nn`. The first level contains branches and the second level contains nodes.

In [29]:
mu.head()

n                 nn  
ID_1_electricity  ID_1    0.475
ID_1_oil          ID_1    0.475
ID_1_K            ID_1    4.050
C_EL_1            EL      0.050
U_ID_1_1          ID_1    0.500
Name: mu, dtype: float64

#### Calculate starting values (corresponding to the solution if the entire tree was Leontief).
For demands/quantities, we use the fact that Leontief demand is given by 
$$q_j = \mu_j \left(\frac{p_i}{p_j}\right)^\sigma q_i = \mu_j q_i$$ for CES demand where $i$ refers to a node and $j$ refers to the branch. The CET and the MNL form is identical under Leontief.\
For prices, we generally use that when we know the price of each branch under a node as well as the quantities of these branches, the zero profit condition can be rearranged to give us the price of the node:
$$ q_i p_i = \sum_j q_j p_j \quad \Leftrightarrow \quad  p_i = \frac{\sum_j q_j p_j}{q_i}$$

Output quantities are held in the `qS` object. Output quantities are held fixed in the partial equilibrum (the output prices are the variables that adjust), and we simply set them to `output_quantity` in this example.

In [30]:
output_quantity = 100
qS = pd.Series([], name="qS", dtype="float64")

`qS` is now an empty series to which we add the value of 10 for each output of the tree:

In [31]:
for out in nt.database.series["out"]:
    qS[out] = output_quantity

All other quantities are kept in the object `qD`.

In [32]:
qD = pd.Series([], name="qD", dtype="float64")

First, we calculate components' starting values for input-displacing (we did it for EOP with qS, because components are outputs there). \
We use the demand equation stated earlier (multiplication of share-parameter and relevant node/component). This calculates quantities for all components, including baseline components

In [33]:
for E in output["ID"]["upper_categories"]:
    for C in output["ID"]["upper_categories"][E]:
        qD[C] = mu.loc[(C, E)] * qS[E]

Next, we use that we have estimates of U to set these (current coverages), and then calculate the $\mu$s residually

In [34]:
for index, curr_coverage in output["ID"]["current_coverages"].iteritems():
    qD[index[0]] = curr_coverage * qS[index[1]]

The $\mu$s of technology goods (in their component nest) residually. This includes the share-parameters for the baseline technology goods $\bar U$. Since we still have not set the starting values for the quantities of these baseline technology goods, we set these using the share-parameters that are calculated here as well.

In [35]:
for C in output["ID"]["components"]:
    #The ':-1' leaves out the baseline technology good, which we handle separately afterwards
    nonbase_U = output["ID"]["components"][C][:-1]
    base_mu = 1
    for U in nonbase_U:
        mu[(U, C)] =  qD[U]/qD[C]
        base_mu -= mu[(U, C)]
    #Quantities of baseline U
    mu[(output["ID"]["components"][C][-1], C)] = base_mu
    qD[output["ID"]["components"][C][-1]] = base_mu * qD[C]
    if base_mu <= 0:
        print(C)
        print(nonbase_U)
        raise Exception("base_mu is not positive")


For end of pipe, we simple set the share-parameters of the technology goods under components as $1/N$ where $N$ is the number of branches/technology goods under a given component. Having set these, we can calculate the corresponding quantities using the CES Leontief demand equation.

In [36]:
for C in output["EOP"]["components"]:
    for U in output["EOP"]["components"][C]:
        mu.loc[(U, C)] = 1/len(output["EOP"]["components"][C])
        qD[U] = mu.loc[(U, C)] * qS[C]

Then, we calculate the starting values of technologies $\tau$ as the sum of their relevant $U$, since the CET function should be scale-preserving.\
The drawback here, for EOP only, is that the quantities of $U$ do not necessarily adhere to the relative sizes of their share-parameters (which are set based on the technology catalog). 

In [37]:
for module in modules:
    for tech in output[module]["techs"]:
        qD[tech] = qD[output[module]["techs"][tech]].sum()

Lastly, we set the starting values of the inputs $X$ that go into technologies. These again use the demand equation. The share-parameters of these were given directly from technology data.

In [38]:
for module in modules:
    techs = output[module]["techs_inputs"]
    for tech in techs:
        for inp in techs[tech]:
            qD[inp] = mu.loc[(inp, tech)] * qD[tech]

The next part contains the setting of starting values for prices.
The prices of inputs (the very bottom of the tree) as fully exogenous, whereas all other prices are endogenous. The prices of outputs (those whose quantities are contained in `qS` are kept in the object `PbT` (refers to "prices before taxes") whereas the rest of the prices are kept in `PwT`.

In [39]:
#Quantity of the IO technology
qD["IO_tech"] = 0
for base in output["ID"]["IO_tech"]["IO_tech"]:
    qD["IO_tech"] += qD[base]
#share parameters for each the non-replacing baseline technologies:
for base in output["ID"]["IO_tech"]["IO_tech"]:
    mu[(base, "IO_tech")] = qD[base] / qD["IO_tech"]

Objects for storing prices:

In [40]:
PwT = pd.Series([], name="PwT", dtype="float64")
PbT = pd.Series([], name="PbT", dtype="float64")

Prices of inputs are also stated in the catalog:

In [41]:
for module in modules:
    for t, inputs in output[module]["techs_inputs"].items():
        for inp in inputs:
            PwT[inp] = output["inputprices"][inp.split("_")[-1]]

Add the prices of technologies: $p^\tau$

In [42]:
for module in modules:
    for t, inputs in output[module]["techs_inputs"].items():
        PwT[t] = pd.concat([PwT[inputs], qD[inputs]], axis=1).product(axis=1).sum() / qD[t]

These prices are equal to unit costs by construction. 
Check:
#CHECK DEVELOPED DONE YET

For $p^U$, we simply assume $p^U = p^\tau$, which, since the share-parameters in the CET nest splitting $\tau$ into its produced technology goods sum to one, is consistent with zero profits.

In [43]:
for module in modules:
    for t in output[module]["techs"]:
        for U in output[module]["techs"][t]:
            PwT[U] = PwT[t]

Prices on components for end of pipe should be allocated to PbT because that includes the prices of outputs:

In [44]:
for C, U_list in output["EOP"]["components"].items():
    PbT[C] = pd.concat([PwT[U_list], qD[U_list]], axis=1).product(axis=1).sum() / qS[C]

Insert baseline input quantities and baseline technology good prices (only relevant for ID since baselines do not exist in EOP).

This also (for now, should be set with input-output macro data later) sets the share-parameters of the inputs for the baseline technology goods. It assumes that the baseline uses all energy inputs in equal proportions and that these share-parameters sum to 1. Using these share-parameters, the corresponding quantities are added to `qD`.

In [45]:
inputs = output["ID"]["IO_tech_inputs"]["IO_tech"]
for inp in inputs:
    mu[(inp, "IO_tech")] = 1/len(inputs)
    PwT[inp] = output["inputprices"][inp.split("_")[-1]]
    qD[inp] = mu[(inp, "IO_tech")] * qD["IO_tech"]
#Price of the IO technology
PwT["IO_tech"] = pd.concat([PwT[inputs], qD[inputs]], axis=1).product(axis=1).sum() / qD["IO_tech"]

In [46]:
for base in output["ID"]["IO_tech"]["IO_tech"]:
    PwT[base] = PwT["IO_tech"] 

Quantities and prices of inputs to replacement-baseline-technologies and the cost of these as well:

In [47]:
for base_U, inputs in output["ID"]["baseline_U_inputs"].items():
    for inp in inputs:
#         mu[(inp, base_U)] = 1/len(inputs)    
        qD[inp] = mu[(inp, base_U)] * qD[base_U]
        PwT[inp] = output["inputprices"][inp.split("_")[-1]]
    PwT[base_U] =  pd.concat([PwT[inputs], qD[inputs]], axis=1).product(axis=1).sum() / qD[base_U]

<!-- Prices of components in ID: -->

In [48]:
for C, U_list in output["ID"]["components"].items():
    PwT[C] = pd.concat([PwT[U_list], qD[U_list]], axis=1).product(axis=1).sum() / qD[C]

Prices of plrea-cebaseline components. \
First, we calculate the share-parameters and corresponding quantities for the inputs that the baseline technology components use. 

In [49]:
# for base_C, inputs in output["ID"]["baseline_C_inputs"].items():
#     for inp in inputs:
#         mu[(inp, base_C)] = 1/len(inputs)
#         qD[inp] = 1/len(inputs) * qD[base_C]
#         PwT[inp] = output["inputprices"][inp.split("_")[-1]]
#     PwT[base_C] =  pd.concat([PwT[inputs], qD[inputs]], axis=1).product(axis=1).sum() / qD[base_C]

Prices of energy services ("upper categories" in ID). 

In [50]:
for E, C_list in output["ID"]["upper_categories"].items():
    PbT[E] = pd.concat([PwT[C_list], qD[C_list]], axis=1).product(axis=1).sum() / qS[E]

We now add the calculated values to the (still) empty database `db` instantiated earlier.

In [51]:
qS.index.name = "n"
qD.index.name = "n"
PwT.index.name = "n"
PbT.index.name = "n"
db["qS"] = qS
db["qD"] = qD
db["PwT"] = PwT
db["PbT"] = PbT
db["mu"] = mu

Merge `db` with the database attached to the nesting tree object `nt`:

In [52]:
DataBase.GPM_database.merge_dbs(nt.database, db, "first")

As a confirmation that this went succesfully, we check that the databse in `nt` contains a symbol called 'mu':

In [53]:
nt.database.series["mu"].vals

n                    nn     
ID_1_electricity     ID_1       0.475
ID_1_oil             ID_1       0.475
ID_1_K               ID_1       4.050
C_EL_1               EL         0.050
U_ID_1_1             ID_1       0.500
                                ...  
IO_tech_electricity  IO_tech    0.200
IO_tech_oil          IO_tech    0.200
IO_tech_inp3         IO_tech    0.200
IO_tech_inp4         IO_tech    0.200
IO_tech_inp5         IO_tech    0.200
Name: mu, Length: 91, dtype: float64

Success! We now proceed to use the class prstatic, a childclass of gmspython.

In [54]:
def flatten_list(list_):
    return [item for sublist in list_ for item in sublist]

## Add sets used for calibration to the database before constructing the model object

#### g_exo_vals consists of $\bar \sigma$ and $\bar \mu$. These are targets in the minimzation object

$\bar \sigma$ parameters used for the minimization object. These are the sigmas in the MNL nests (C->U)

In [55]:
sigmabar = pd.Series([], name="sigmabar", dtype="float64")
for c in output["ID"]["components"].keys():
    sigmabar[c] = 1

$\bar \mu$ parameters used for minimization object. These are the share parameters in the CET split from technologies to technology goods.

In [56]:
mubar = mu.loc[pd.IndexSlice[mu.index.get_level_values(0).isin(flatten_list(list(output["ID"]["techs"].values()))), \
                     mu.index.get_level_values(1).isin(list(output["ID"]["techs"].keys()))]]
mubar.name = "mubar"

#### g_tech_endo consists of parameters that are endogenized in the calibration procedure

In [57]:
mu_EC = mu.loc[pd.IndexSlice[pd.Series(mu.index.get_level_values(0).str.startswith("C")), \
                     pd.Series(mu.index.get_level_values(1).str.startswith("E"))]].index

In [58]:
mu_IOtech = mu.loc[pd.IndexSlice[mu.index.get_level_values(0).str.startswith("IO"), mu.index.get_level_values(0).str.startswith("IO")]].index

In [59]:
sigma_CU = sigmabar.index
sigma_CU.name = "n"

In [60]:
mu_CU = mu.loc[mu.index.get_level_values(0).isin(flatten_list(output["ID"]["components"].values())), mu.index.get_level_values(1).isin(output["ID"]["components"].keys())].index
mu_CU.name = "n"

In [61]:
mu_tautoU = mu.loc[pd.IndexSlice[mu.index.get_level_values(0).str.startswith("U"), mu.index.get_level_values(1).isin(output["ID"]["techs"].keys())]].index

Share-parameters of Leontief technologies (to be kept exogenous throughout)

In [62]:
t = pd.IndexSlice[pd.Series(mu.index.get_level_values(1).isin(output["ID"]["techs"].keys())), pd.Series(mu.index.get_level_values(1).isin(output["ID"]["techs"].keys()))]

In [63]:
t2 = pd.IndexSlice[pd.Series(mu.index.get_level_values(1).isin(output["ID"]["baseline_U_inputs"].keys())), \
                   pd.Series(mu.index.get_level_values(1).isin(output["ID"]["baseline_U_inputs"].keys()))]

In [64]:
mu_leontief_techs = mu.loc[t2].index.append(mu.loc[t].index)

In [65]:
tech_endo_mu = mu_EC.append(mu_IOtech).append(mu_CU).append(mu_tautoU)

In [66]:
tech_endo_sigma = sigma_CU

g_endovars_exoincalib includes the endogenous variables that we exogenize in calibration because we have data on them (C due to potentials, sum of U due to current applications, sum of dX for IO tech and replacing tech due to IO data) 

In [67]:
def multiindex_series(idx_level_names, idx_name=None, series_name=None):
    if idx_name is None and series_name is not None:
        idx_name = series_name
    elif idx_name is not None and series_name is None:
        series_name = idx_name
    elif idx_name is None and series_name is None:
        raise Exception("Supply either index name or series name")
    idx = pd.MultiIndex(levels=[[]]*len(idx_level_names), codes=[[]]*len(idx_level_names), names=idx_level_names)
    ser = pd.Series(index=idx, dtype=float)
    ser.rename(series_name, inplace=True)
    #ser.index.name = idx_name
    return ser

In [68]:
def find_key_from_value(d, value):
    assert isinstance(d, dict)
    out = []
    for (k, v) in d.items():
        if value in v:
            out.append(k)
    if len(out) == 1:
        return out[0]
    else:
        raise Exception("Value exists in multiple keys")

In [69]:
sumU = multiindex_series(idx_level_names=["n", "nn"], series_name="sumU")
for t in output["ID"]["techs"]:
    for U in output["ID"]["techs"][t]:
        C = find_key_from_value(output["ID"]["components"], U)
        E = find_key_from_value(output["ID"]["upper_categories"], C)
        sumU[("sumU_" + t + "_" + E, U)] = qD[U]

In [70]:
sumU_map = sumU.index

In [71]:
sumU_calibvalues = sumU.copy()
sumU_calibvalues.index = sumU_calibvalues.index.droplevel(1)
sumU_calibvalues = sumU_calibvalues.groupby("n").sum()

In [72]:
def find_true_input(inp, Q2P):
    true = list(Q2P[Q2P.get_level_values(0).isin([inp])].get_level_values(1))[0]
    return true

In [73]:
#sumX. The sum of energy use for baseline technologies (IO_tech + replacing baselines)
sumX = multiindex_series(idx_level_names=["n", "nn"], series_name="sumX")
for x in output["ID"]["IO_tech_inputs"]["IO_tech"]:
    inp = find_true_input(x, output["ID"]["Q2P"])
    sumX[(inp, x)] = np.nan
#pd.IndexSlice[pd.Series(mu.index.get_level_values(1).isin(output["ID"]["techs"].keys())), pd.Series(mu.index.get_level_values(1).isin(output["ID"]["techs"].keys()))]
#output["ID"]["Q2P"]

for t in output["ID"]["techs_inputs"]:
    for x in output["ID"]["techs_inputs"][t]:
        sumX[(find_true_input(x, output["ID"]["Q2P"]), x)] = np.nan

for baseU in output["ID"]["baseline_U_inputs"]:
    for x in output["ID"]["baseline_U_inputs"][baseU]:
        sumX[(find_true_input(x, output["ID"]["Q2P"]), x)] = np.nan

#change name of the sum of an input to "sum_x"
sumX = sumX.reset_index()
sumX["n"] = "sum_" + sumX["n"]
sumX = sumX.set_index(["n", "nn"])["sumX"].index

g_endovars_exoincalib

In [74]:
endovars_exoincalib_sumU = sumU_map.copy()
endovars_exoincalib_sumX = sumX
endovars_exoincalib_C = mu_EC.get_level_values(0)

In [75]:
qD_exo_U =  mu_tautoU.get_level_values(0)

In [76]:
calib_values_currentapplications = pd.concat([qD[qD.index.str.startswith("U") & ~qD.index.str.contains("EOP")], qD[qD.index.str.startswith("C")]]) 

In [77]:
calib_values_potentials = mu.loc[pd.IndexSlice[pd.Series(mu.index.get_level_values(0).str.startswith("C")), \
                                 pd.Series(mu.index.get_level_values(1).str.startswith("E"))]]

In [78]:
db = DataBase.GPM_database()
alwaysexo_mu = mu.drop(tech_endo_mu).index

In [79]:
db["exo_parameters_mu"] = mubar
db["exo_parameters_sigma"] = sigmabar
db["tech_endoincalib_sigma"] = tech_endo_sigma
db["tech_endoincalib_mu"] = tech_endo_mu
db["params_alwaysexo_mu"] = alwaysexo_mu
db["endovars_exoincalib_sumU"] = endovars_exoincalib_sumU
db["endovars_exoincalib_sumX"] = endovars_exoincalib_sumX
db["endovars_exoincalib_C"] = endovars_exoincalib_C
db["calib_values_currentapplications"] = calib_values_currentapplications
db["calib_values_potentials"] = calib_values_potentials
db["obj"] = 1

In [80]:
db["sumUaggs"] = endovars_exoincalib_sumU.get_level_values(0).unique()
db["sumU2U"] = endovars_exoincalib_sumU
db["qsumU"] = sumU_calibvalues

In [81]:
db["sumXaggs"] = endovars_exoincalib_sumX.get_level_values(0).unique()
db["sumX2X"] = endovars_exoincalib_sumX
db["qsumX"] = pd.Series([10]*len(endovars_exoincalib_sumX.get_level_values(0).unique()), index=endovars_exoincalib_sumX.get_level_values(0).unique(), name="qsumX")

# **2.1: The model**
### **2.1.1 setup**

*Text adopted from the code test_class:*

Using the nesting trees above that defines various subsets, we now build a gams model using a *gmspython* class (pr_static). We build the partial equilibrium model from the following principles:
* The firm takes input prices as given. These are prices with taxes ('PwT') that are defined over the global set *inp*. In this case this includes e.g. the price on electricity and capital.
* The firm further takes the quantity of supply as given (it has to be either this or output prices for the model to be square). We denote the quantity of supply 'qS', and define it over the subset of goods that are outputs from the sector *out*. 
* The firm's demand for intermediate goods, and their prices are endogenous to the firm: In our case this involves $\tau$, $U$ and $C$ in ID and only $\tau$ and $U$ in EOP. These intermediate goods are not traded on any market, but is simply a construct we use to build the nests. In this way the quantity of $U$ is both supplied, and demanded by the firm/sector itself. As we do not need both a supply and demand variable (they are the same in this case), we let the quantity/prices for intermediate goods go under the variables 'qD'/'PwT', defined over the subset *int*.
* The demand for inputs is also endogenous to the firm. We denote this 'qD' defined over the subset of goods 'inp'.
* As briefly mentioned above, we either have to let the output prices / quantities be endogeonous (and the other exogenous) for the module to be square. In this case we let the price on outputs be endogenous. We denote the price 'PbT' for price before taxes. This is defined over the subset *out*. Distinguishing between PbT and PwT allows for the flexibility of including a tax module / including a mark-up on profits at a later point.

We will get back to the way we initialize the *gmspython* module next; here we initialize the class to be able to print the variables needed for this specific model.
The model, denoted `m`, is an instance of the class `pr_static` which is itself a childclass of the more general `gmspython` class.

#### Construct sets used specifically for the abate class:

In [82]:
nt.database.series.__dict__

{'name': 'Abatement',
 'database': {'alias_': <DataBase.gpy_symbol at 0x1fdc4b4d808>,
  'alias_set': <DataBase.gpy_symbol at 0x1fdc4b4db48>,
  'alias_map2': <DataBase.gpy_symbol at 0x1fdc4b4dc88>,
  'n': <DataBase.gpy_symbol at 0x1fdc4b4d548>,
  'map_all': <DataBase.gpy_symbol at 0x1fdc4b28248>,
  'inp': <DataBase.gpy_symbol at 0x1fdc4bf5248>,
  'out': <DataBase.gpy_symbol at 0x1fdc4bf52c8>,
  'int': <DataBase.gpy_symbol at 0x1fdc4bf5608>,
  'fg': <DataBase.gpy_symbol at 0x1fdc4bf5308>,
  'wT': <DataBase.gpy_symbol at 0x1fdc4bf5648>,
  'kno_out': <DataBase.gpy_symbol at 0x1fdc4bf50c8>,
  'kno_inp': <DataBase.gpy_symbol at 0x1fdc4bf5348>,
  'qS': <DataBase.gpy_symbol at 0x1fdc4bf5388>,
  'qD': <DataBase.gpy_symbol at 0x1fdc4bf53c8>,
  'PwT': <DataBase.gpy_symbol at 0x1fdc4b4d8c8>,
  'PbT': <DataBase.gpy_symbol at 0x1fdc4b4d2c8>,
  'mu': <DataBase.gpy_symbol at 0x1fdc4c34148>}}

Values to calibrate to. Potentials give C/E (i.e. C) and current application gives U/E (i.e. U) (IO data gives dX under IO_tech, and is loaded later)

In [83]:
m = abatement.abate(nt=nt, tech_db=db, work_folder=work_folder, **{'data_folder': gams_folder, "name":"Abatement"})
m.model.database.update_all_sets(clean_up=False) #Makes n include the sum variables

In [84]:
m.model.database.alias_dict0

{'n': Index(['n', 'nn', 'nnn'], dtype='object', name='alias_map2')}

In [85]:
m.get("nn")

Index(['C_EL_3', 'U_ID_5_2', 'sum_inp3', 'C_SO2_1', 'U_ID_C_EL_3_base_inp4',
       'sumU_ID_5_ER', 'EOP_t1_K', 'U_ID_C_EL_3_base', 'sumU_ID_2_EL',
       'U_EOP_t1_1', 'EOP_t1_oil', 'ID_2_K', 'ID_1_K', 'IO_tech_electricity',
       'U_ID_5_1', 'sum_oil', 'U_ID_C_EL_4_base', 'C_EL_4', 'C_ER_base', 'EH',
       'U_ID_4_1', 'U_ID_C_EL_1_base_oil', 'ID_1_oil', 'U_ID_1_3', 'ER',
       'U_ID_4_2', 'ID_1', 'U_ID_2_1', 'sumU_ID_4_ER', 'U_ID_C_EL_5_base',
       'sumU_ID_3_EL', 'C_EL_2', 'ID_4', 'EOP_t1', 'IO_tech_oil', 'ID_5_oil',
       'IO_tech_inp3', 'C_EH_2', 'C_EL_base', 'C_EH_1', 'ID_5', 'IO_tech',
       'C_ER_2', 'ID_3_inp4', 'ID_3_K', 'sumU_ID_2_EH', 'ID_4_electricity',
       'IO_tech_inp5', 'U_ID_C_ER_1_base', 'U_ID_3_1', 'U_ID_C_EH_2_base',
       'U_ID_4_3', 'ID_4_inp5', 'U_ID_C_EL_2_base_inp3', 'U_ID_C_EL_2_base',
       'sumU_ID_4_EH', 'U_ID_C_EL_2_base_oil', 'sumU_ID_5_EH',
       'U_ID_C_EL_3_base_inp3', 'ID_2_oil', 'ID_4_inp3', 'U_ID_2_2',
       'ID_2_electricity', 'ID_5_K

In [86]:
m.model.database.sets

{'sets': ['alias_set', 'alias_map2', 'n', 'nn', 'nnn'],
 'subsets': ['inp',
  'out',
  'int',
  'fg',
  'wT',
  'kno_out',
  'kno_inp',
  'kno_ID_EC',
  'bra_ID_EC',
  'inp_ID_EC',
  'out_ID_EC',
  'kno_no_ID_EC',
  'bra_o_ID_EC',
  'bra_no_ID_EC',
  'kno_ID_CU',
  'bra_ID_CU',
  'inp_ID_CU',
  'out_ID_CU',
  'kno_no_ID_CU',
  'bra_o_ID_CU',
  'bra_no_ID_CU',
  'kno_ID_TU',
  'bra_ID_TU',
  'inp_ID_TU',
  'out_ID_TU',
  'bra_o_ID_TU',
  'bra_no_ID_TU',
  'kno_ID_TX',
  'bra_ID_TX',
  'inp_ID_TX',
  'out_ID_TX',
  'kno_no_ID_TX',
  'bra_o_ID_TX',
  'bra_no_ID_TX',
  'kno_ID_IOCU',
  'bra_ID_IOCU',
  'inp_ID_IOCU',
  'out_ID_IOCU',
  'bra_o_ID_IOCU',
  'bra_no_ID_IOCU',
  'kno_ID_IOX',
  'bra_ID_IOX',
  'inp_ID_IOX',
  'out_ID_IOX',
  'kno_no_ID_IOX',
  'bra_o_ID_IOX',
  'bra_no_ID_IOX',
  'kno_ID_UbaseX',
  'bra_ID_UbaseX',
  'inp_ID_UbaseX',
  'out_ID_UbaseX',
  'kno_no_ID_UbaseX',
  'bra_o_ID_UbaseX',
  'bra_no_ID_UbaseX',
  'kno_EOP_CU',
  'bra_EOP_CU',
  'inp_EOP_CU',
  'out_EOP_CU'

Recall that running a model requires constructing three methods/properties for `m`:

1. `m.initialize_variables()`
2. `m.endo_groups` and `m.exo_groups`
3. `m.add_blocks()`

We briefly review each in turn.

`m.initialize_variables()` sets default initial values for the variables and parameters that are not already present in the attached database (which we constructed earlier). For example, we did not specify the values for the mark-up parameters anywhere, so the method `initialize_variables()` *must* specify the value that these parameters should have. We can see that in this case they will simply be set to 1 (no markup):

In [87]:
# print(m.group_conditions("qD_exo_U"))
m.initialize_variables(check_variables=True)
#group = "g_qD_exo_U"

In [88]:
m.group_conditions("g_prices_exoincalib")

[{'PbT': {'and': [<DataBase.gpy_symbol at 0x1fdc4c29e48>,
    {'not': <DataBase.gpy_symbol at 0x1fdc4c62648>}]},
  'Peq': <DataBase.gpy_symbol at 0x1fdc4c06a48>}]

In [89]:
m.default_var_series("mark_up")

We run the method using the keyword argument `check_variables:True`. This makes the program check whether the database contains all the necessary values of a specific variable/parameter (say, $\mu$), and if not, merges the default value (in this case equal to 1) onto the series of share-parameters already contained in the database. 

In [90]:
m.initialize_variables(**{"check_variables":True})

Since all of our starting values were set 'as if' the entire tree was Leontief, it is worth noting that the default values for substitutions of elasticity/transformation have the following values:

In [91]:
### INSERT SHOWING VALUES

The second requirement(s) are the properties `endo_groups` and `exo_groups`. These collect the variables in groups and specify which of these subgroups are part of the two upper groups of endogenous and exogenous variables/parameters respectively. The choice of whether a variable should be endogenous or exogenous changes e.g. depending on whether the model is about to be solved for calibration purposes or not.
In the basic case (not calibration mode), the endogenous variables are split into two groups:

In [92]:
m.endo_groups.keys()

dict_keys(['Abatement_g_prices_alwaysendo', 'Abatement_g_quants_alwaysendo', 'Abatement_g_prices_exoincalib', 'Abatement_g_quants_exoincalib'])

... which themselves include the actual variables. Here we print the values of prices with taxes in the endovars group.

In [93]:
print(m.endo_groups["Abatement_g_quants_alwaysendo"][0])
print(m.endo_groups["Abatement_g_quants_alwaysendo"][0]["conditions"])

{'name': 'qD', 'conditions': [{'and': [<DataBase.gpy_symbol object at 0x000001FDC4C29B08>, <DataBase.gpy_symbol object at 0x000001FDC4C29A88>, {'not': <DataBase.gpy_symbol object at 0x000001FDC4C51788>}]}]}
[{'and': [<DataBase.gpy_symbol object at 0x000001FDC4C29B08>, <DataBase.gpy_symbol object at 0x000001FDC4C29A88>, {'not': <DataBase.gpy_symbol object at 0x000001FDC4C51788>}]}]


And next we print the exogenous prices with taxes by doing the same thing, but for g_exovars instead. This is found in the exo_groups attribute rather than the endo_groups attribute:

Finally, we need to specify the *add_blocks* method. This is the method that writes blocks of equations to the model. In our case, we write a different block of equations for each tree.
For example, a set of equations are:

In [94]:
print(m.eqtext(list(m.ns_local)[0]))

E_zp_out_ID_EC[n]$(out_ID_EC[n])..	PbT[n]*qS[n] =E= sum(nn$(map_ID_EC[nn,n]), qD[nn]*PwT[nn]);
	E_zp_nout_ID_EC[n]$(kno_no_ID_EC[n])..	PwT[n]*qD[n] =E= sum(nn$(map_ID_EC[nn,n]), qD[nn]*PwT[nn]);
	E_q_out_ID_EC[n]$(bra_o_ID_EC[n])..	qD[n] =E= sum(nn$(map_ID_EC[n,nn]), mu[n,nn] * (PbT[nn]/PwT[n])**(sigma[nn]) * qS[nn] / sum(nnn$(map_ID_EC[nnn,nn]), mu[nnn,nn] * (PbT[nn]/PwT[nnn])**(sigma[nn])));
	E_q_nout_ID_EC[n]$(bra_no_ID_EC[n])..	qD[n] =E= sum(nn$(map_ID_EC[n,nn]), mu[n,nn] * (PwT[nn]/PwT[n])**(sigma[nn]) * qD[nn] / sum(nnn$(map_ID_EC[nnn,nn]), mu[nnn,nn] * (PwT[nn]/PwT[nnn])**(sigma[nn])));


In [95]:
m.add_blocks()

In [96]:
m.simplesum.__dict__

{'n': <DataBase.gpy_symbol at 0x1fdc4c29a48>,
 'sumUaggs': <DataBase.gpy_symbol at 0x1fdc4c51288>,
 'sumU2U': <DataBase.gpy_symbol at 0x1fdc4c51308>,
 'qsumU': <DataBase.gpy_symbol at 0x1fdc4c51588>,
 'sumXaggs': <DataBase.gpy_symbol at 0x1fdc4c51208>,
 'sumX2X': <DataBase.gpy_symbol at 0x1fdc4c512c8>,
 'qsumX': <DataBase.gpy_symbol at 0x1fdc4c510c8>,
 'qD': <DataBase.gpy_symbol at 0x1fdc4c5d8c8>,
 'aliases': {0: 'n', 1: 'nn', 2: 'nnn'},
 'conditions': {'sumUaggs': 'sumUaggs[n]', 'sumXaggs': 'sumXaggs[n]'}}

In [97]:
m.add_blocks()

In [98]:
c = m.simplesum.add_symbols(m.model.database, m.ns)

In [99]:
m.simplesum.n.name

'n'

In [100]:
m.model.database.alias_dict0

{'n': Index(['n', 'nn', 'nnn'], dtype='object', name='alias_map2')}

In [101]:
m.simplesum.aliases

{0: 'n', 1: 'nn', 2: 'nnn'}

In [102]:
m.ns_local.keys()

dict_keys(['ID_EC', 'ID_CU', 'ID_TU', 'ID_TX', 'ID_IOCU', 'ID_IOX', 'ID_UbaseX', 'EOP_CU', 'EOP_TU', 'EOP_TX'])

In [103]:
m.model.blocks.keys()

dict_keys(['M_ID_EC', 'M_ID_CU', 'M_ID_TU', 'M_ID_TX', 'M_ID_IOCU', 'M_ID_IOX', 'M_ID_UbaseX', 'M_EOP_CU', 'M_EOP_TU', 'M_EOP_TX', 'M_Abatement_simplesum'])

To sum up, the model we have in mind here has the following main settings (where a \$ denotes a condition):
* Endogenous variables: $PwT\$(int)$, $PbT\$(out)$, $qD\$(wT)$. (Recall that the subset wT is the union of intermediate goods and inputs)
* Exogenous variables: $PwT\$(inp)$, $qS\$(out)$. 
* Equations: For the CES nest we have the following two equations:
    $$\begin{align}
        q_j =& \mu_j\left(\dfrac{p_j}{p_i}\right)^{-\sigma}q_i, \tag{CES-1}\\
        p_iq_i =& \sum_j q_jp_j \tag{CES-2}
    \end{align}$$
    (CES-1) is the CES demand function that has to hold for all *branches* ($q_j$) where $q_i$ is the relevant knot in the nesting tree. (CES-2) is a zero-profit condition that has to hold for production functions with constant returns to scale (alternatively, we can use a price index). This has to hold for every *knot* where $j$ sums over the relevant branches. A similar thing has to hold for the CET tree.

### **2.1.2 Running the model**
Running the model, including the invoking the methods and properties described in the previous section, we simply run the following:

In [104]:
m.add_groups()

In [105]:
m.write_and_run(kwargs_init={'check_variables':True})

To check whether the model was successfully solved, we check the modelstat (16.0 means solved correctly, 5.0 means not)

In [106]:
m.model_instances["baseline"].modelstat

5.0

Next, we wish to solve the model while instead of the default value of substitution and transformation elasticities of 0.1, we set them equal to zero (in practice not exactly, but very close).
Since the starting values have been set under the assumption of Leontief nests, this will help the solver find an initial solution.

In [107]:
m.ns_local.keys()

dict_keys(['ID_EC', 'ID_CU', 'ID_TU', 'ID_TX', 'ID_IOCU', 'ID_IOX', 'ID_UbaseX', 'EOP_CU', 'EOP_TU', 'EOP_TX'])

In [108]:
output["ID"].keys()

dict_keys(['techs_inputs', 'techs', 'components', 'upper_categories', 'mu', 'Q2P', 'unit_costs', 'current_coverages', 'coverage_potentials', 'IO_tech', 'IO_tech_inputs', 'baseline_U_inputs'])

In [109]:
m.model.settings.databases["Abatement_0"]["sigma"].vals.loc[:] = 0.0001
m.model.settings.databases["Abatement_0"]["eta"].vals.loc[:] = -0.0001
#m.model.settings.databases["Abatement"]["sigma"].vals[condition] = 0.2
#m.write_and_run(options_run={'output':sys.stdout})
m.write_and_run()
if m.model_instances["baseline"].modelstat == 16.0:
    print("\nSuccess! The modelstat was 16.0")


Success! The modelstat was 16.0


In [110]:
m.get("qsumX")

n
sum_electricity    10
sum_oil            10
sum_inp3           10
sum_inp4           10
sum_inp5           10
sum_K              10
Name: qsumX, dtype: int64

The model could be solved using elasticities of 0.0001. Let's check if it can be solved when these are set to higher numbers, implying a solution farther from the starting values (which, as mentioned, were set based on Leontief).

In [105]:
# condition = m.model.settings.databases["Abatement"]["sigma"].vals.index.str.startswith("C_E") & ~m.model.settings.databases["Abatement"]["sigma"].vals.index.str.contains("base")

In [111]:
for sigma_val in range(0, 60, 10):
    sigma_val = sigma_val/100
    if sigma_val == 0:
        sigma_val = 0.0001
#    for MNL_sigma in [0.1, 1]:

    #Set values
    #m.model.settings.databases["Abatement"]["sigma"].vals.loc[condition] = MNL_sigma
    #m.model.settings.databases["Abatement"]["sigma"].vals.loc[~condition] = sigma_val
    m.model.settings.databases["Abatement_0"]["sigma"].vals.loc[:] = sigma_val
    m.model.settings.databases["Abatement_0"]["eta"].vals.loc[:] = -sigma_val

    #Try to solve
    m.write_and_run()

    #How did it go?
    if m.model_instances["baseline"].modelstat == 16.0:
        print("\nSUCCESS:")
        print("Sigma/eta:", round(sigma_val, 4))
        #print("Sigma/eta (MNL):", round(MNL_val, 4))
    else:
        print("\nFAIL:")
        print("Sigma/eta:", round(sigma_val, 4))
        #print("Sigma/eta: (MNL)", round(MNL_sigma, 4))


SUCCESS:
Sigma/eta: 0.0001

SUCCESS:
Sigma/eta: 0.1

SUCCESS:
Sigma/eta: 0.2

SUCCESS:
Sigma/eta: 0.3

FAIL:
Sigma/eta: 0.4

FAIL:
Sigma/eta: 0.5


As can be seen, the model cannot be solved when the elasticities get too large. To accomodate this, we replace each iteration's starting values with the previous iteration's solution, while gradually increasing $\sigma$ and $\mu$.
This allows us to set these to (hopefully) any value. This "gradual changing of parameters" is called 'sneaky solve'. :)

### Sneaky solve: Gradual increase in sigma

In [103]:
db_star = DataBase.GPM_database()

In [104]:
sigma_star = m.get("sigma").copy()
eta_star = m.get("eta").copy()

In [105]:
sigma_star[output["ID"]["upper_categories"].keys()] = 0.01
sigma_star[output["ID"]["components"].keys()] = 1.2
sigma_star[output["ID"]["baseline_U_inputs"].keys()] = 0.01
sigma_star[output["ID"]["baseline_C_inputs"].keys()] = 0.01
sigma_star[output["ID"]["techs"].keys()] = 0.01
eta_star[output["ID"]["techs"].keys()] = -0.9

KeyError: 'baseline_C_inputs'

In [ ]:
db_star["sigma"] = sigma_star
db_star["eta"] = eta_star

Run again, while adding a checkpoint that we can use afterwards

In [133]:
m.checkpoints

{}

In [134]:
m.model_instances

{'baseline': <DB2Gams_l3.gams_model at 0x23a06a3b9c8>}

In [135]:
m.model.settings.databases["Abatement_0"]["sigma"].vals.loc[:] = 0.001
m.model.settings.databases["Abatement_0"]["eta"].vals.loc[:] = -0.001
name = "calibrated"
#Fejler med noget "delimiter" fejl
#m.write_and_run(name="test", add_checkpoint="test", options_add={"checkpoint":m.checkpoints["leontief"]})
m.write_and_run(name=name, add_checkpoint="leontief")

In [136]:
m.checkpoints[name] = m.model_instances[name].ws.add_checkpoint()

In [137]:
m.checkpoints

{'leontief': <gams.execution.GamsCheckpoint at 0x23a06b82d48>,
 'calibrated': <gams.execution.GamsCheckpoint at 0x23a07b9c848>}

Solve sneakily, gradually increasing sigmas towards their desired values (which themselves are stated in `db_star`)

In [138]:
#m.model_instances[name].solve_sneakily(db_star=db_star, options_run={"checkpoint":name})
#DETTE OPDATERER IKKE CHECKPOINTET "calibrated". VIRKER OGSÅ SPØJST AT MAN SKAL GIVE ET CHECKPOINT NÅR NU IDEEN ER AT DEN SKAL OPRETTE ET BASERET PÅ HVOR DEN ENDER.
# OG JEG HAR JO ALLEREDE GIVET DEN INIT-tingene. 

#gm.checkpoints[name] = mi.ws.add_checkpoint()
m.model_instances[name].solve_sneakily(db_star=db_star, from_cp=True, cp_init=m.checkpoints["leontief"], options_run={"checkpoint":m.checkpoints[name]})

{'Modelstat': 16.0, 'Solvestat': 1.0}

In [139]:
mi = m.model_instances[name]
# newjob = mi.ws.add_job_from_string("sets a,b;",**{'checkpoint': m.checkpoints[name]})
# newjob.run()

Check if the model ran succesfully:

In [150]:
m.model_instances[name].out_db.series["Abatement_B_modelstat"].vals

16.0

The changes in e.g. demanded quantities, from the initial point (the solution where sigmas were ~0) are:

In [151]:
m.model_instances[name].out_db["qD"].vals - m.get("qD")

n
C_EH_1                         -0.000214
C_EH_2                         -0.000121
C_EH_3                         -0.000875
C_EH_base                       0.001219
C_EH_base_K                     0.019945
                                  ...   
U_ID_C_ER_2_base_electricity    0.165325
U_ID_C_ER_2_base_inp3           0.165325
U_ID_C_ER_2_base_inp4           0.165325
U_ID_C_ER_2_base_inp5           0.160729
U_ID_C_ER_2_base_oil            0.165325
Name: qD, Length: 140, dtype: float64

while demand itself ended in the following quantities:

In [152]:
m.model_instances[name].out_db["qD"].vals

n
C_EH_1                          14.999786
C_EH_2                           2.999879
C_EH_3                           4.999125
C_EH_base                       77.001219
C_EH_base_K                     12.853278
                                  ...    
U_ID_C_ER_2_base_electricity     0.665325
U_ID_C_ER_2_base_inp3            0.665325
U_ID_C_ER_2_base_inp4            0.665325
U_ID_C_ER_2_base_inp5            0.660729
U_ID_C_ER_2_base_oil             0.665325
Name: qD, Length: 140, dtype: float64

And let's confirm that our sigmas actually ended in the desired spots:

In [153]:
m.model_instances[name].out_db["sigma"].vals

n
C_CO2_1             0.50
C_EH_1              1.20
C_EH_2              1.20
C_EH_3              1.20
C_EH_base           0.01
C_EL_1              1.20
C_EL_2              1.20
C_EL_3              1.20
C_EL_4              1.20
C_EL_5              1.20
C_EL_base           0.01
C_ER_1              1.20
C_ER_2              1.20
C_ER_base           0.01
EH                  0.01
EL                  0.01
EOP_tau1            0.50
ER                  0.01
ID_1                0.01
ID_2                0.01
ID_3                0.01
ID_4                0.01
ID_5                0.01
U_ID_C_EH_1_base    0.01
U_ID_C_EH_2_base    0.01
U_ID_C_EH_3_base    0.01
U_ID_C_EL_1_base    0.01
U_ID_C_EL_2_base    0.01
U_ID_C_EL_3_base    0.01
U_ID_C_EL_4_base    0.01
U_ID_C_EL_5_base    0.01
U_ID_C_ER_1_base    0.01
U_ID_C_ER_2_base    0.01
Name: sigma, dtype: float64

## A shock to the oil price
We present graphically what happens when the oil price gradually increases (static model solution for each price).
For this, we again use the sneaky solve, but we make sure to extract variables of interest for each iteration, in contrast to above where we were not interested in the solutions per se, but simply that the values of $\sigma$ ended in the desired spots.

In [121]:
#mideleritid start fra starten
#m.model.settings.databases["Abatement_0"]["sigma"].vals.loc[:] = 0.1
#m.model.settings.databases["Abatement_0"]["eta"].vals.loc[:] = -0.1

In [122]:
# m.write_and_run(name="calibrated", add_checkpoint="calibrated")

In [123]:
m.model_instances["calibrated"].out_db.get("sigma")

n
C_CO2_1             0.50
C_EH_1              1.20
C_EH_2              1.20
C_EH_3              1.20
C_EH_base           0.01
C_EL_1              1.20
C_EL_2              1.20
C_EL_3              1.20
C_EL_4              1.20
C_EL_5              1.20
C_EL_base           0.01
C_ER_1              1.20
C_ER_2              1.20
C_ER_base           0.01
EH                  0.01
EL                  0.01
EOP_tau1            0.50
ER                  0.01
ID_1                0.01
ID_2                0.01
ID_3                0.01
ID_4                0.01
ID_5                0.01
U_ID_C_EH_1_base    0.01
U_ID_C_EH_2_base    0.01
U_ID_C_EH_3_base    0.01
U_ID_C_EL_1_base    0.01
U_ID_C_EL_2_base    0.01
U_ID_C_EL_3_base    0.01
U_ID_C_EL_4_base    0.01
U_ID_C_EL_5_base    0.01
U_ID_C_ER_1_base    0.01
U_ID_C_ER_2_base    0.01
Name: sigma, dtype: float64

In [124]:
out_db = m.model_instances["calibrated"].out_db

In [125]:
m.write_and_run(name="oilshock")
#db_out = m.model_instances["baseline"].out_db

In [126]:
#m.write_and_run(name='taxshock', add_checkpoint='taxshock', overwrite=True)
mi = m.model_instances['oilshock']

In [127]:
#mi.out_db.series.__dict__

*2: Define structure of shock* 

In [128]:
db_end_of_loop = DataBase.GPM_database()

In [129]:
cond = out_db.get("PwT").index.str.contains("oil")
db_end_of_loop["PwT"] = out_db.get("PwT")[cond] + 12
#end_prices[cond] = out_db.get("PwT")[cond] + 5
#end_prices = end_prices[cond]

In [130]:
db_end_of_loop.series["PwT"].vals

n
C_EH_base_oil           13.0
C_EL_base_oil           13.0
C_ER_base_oil           13.0
ID_1_oil                13.0
ID_2_oil                13.0
ID_5_oil                13.0
U_ID_C_EH_1_base_oil    13.0
U_ID_C_EH_2_base_oil    13.0
U_ID_C_EH_3_base_oil    13.0
U_ID_C_EL_1_base_oil    13.0
U_ID_C_EL_2_base_oil    13.0
U_ID_C_EL_3_base_oil    13.0
U_ID_C_EL_4_base_oil    13.0
U_ID_C_EL_5_base_oil    13.0
U_ID_C_ER_1_base_oil    13.0
U_ID_C_ER_2_base_oil    13.0
Name: PwT, dtype: float64

In [131]:
#NY ME DDIREKTE
#m.model_instances["oilshock"].solve_sneakily(db_star=db_end_of_loop, from_cp=True, cp_init=m.checkpoints["calibrated"])

In [132]:
(shock_db, shock_kwargs) = ShockFunction.sneaky_db(out_db, db_end_of_loop, n_steps = 25, loop_name='oil_loop')

*3: Define the solution we wish to store for each iteration (conditions argument is optional, but here we extract only the production sector goods):*

In [133]:
shock_db.series["PwT_oil_loop_subset"].vals

Index(['C_EH_base_oil', 'C_EL_base_oil', 'C_ER_base_oil', 'ID_1_oil',
       'ID_2_oil', 'ID_5_oil', 'U_ID_C_EH_1_base_oil', 'U_ID_C_EH_2_base_oil',
       'U_ID_C_EH_3_base_oil', 'U_ID_C_EL_1_base_oil', 'U_ID_C_EL_2_base_oil',
       'U_ID_C_EL_3_base_oil', 'U_ID_C_EL_4_base_oil', 'U_ID_C_EL_5_base_oil',
       'U_ID_C_ER_1_base_oil', 'U_ID_C_ER_2_base_oil'],
      dtype='object', name='n')

In [134]:
store_sol = {'qD': {'domains': shock_kwargs['loop_name']}}

*4: Create Shock instance*

In [135]:
shock = mi.std_UEVAS_from_db(shock_db, loop_name=shock_kwargs['loop_name'], shock_name=shock_kwargs['shock_name'], store_sol=store_sol)
#shock = mi.std_UEVAS_from_db(shock_db, loop_name=shock_kwargs['loop_name'], shock_name=shock_kwargs['shock_name'])
#solve_sneakily skal have selve checkpointet
#Gør noget så sneaky solve laver et sidste checkpoint

*5: Execute shock from the 'calibrated' checkpoint i.e. the model state after the sigmas have been chosen:*

In [136]:
mi.execute_shock_from_cp(shock=shock, cp=m.checkpoints['calibrated'])

In [137]:
sol_qD = mi.out_db.series["sol_qD"].vals

In [138]:
#tech_names = ["ID_thighest", "ID_thigher", "ID_tmedium", "ID_tlower", "ID_tlowest"]
#sol_qD.loc(axis=0)["oil_loop_1", tech_names]

In [139]:
sol_qD.reset_index().to_excel(os.getcwd() + "\\oil_price_shock_" + inputfile.split("_")[-1], index=False)

In [140]:
#m.model_instances["oilshock"].solve_sneakily(db_star=db_end_of_loop, from_cp=True, cp_init=m.checkpoints["oilshock"])

The solution is stored as a parameter noted as "sol_"+ the name of the variable. In our case, the specific quantities are stored:

### **3.1 Diagnostics (dated)**
When the model cannot be solved, we run some diagnostics:

In [ ]:
# for symbol in ["PbT", "PwT", "qD"]:
#     start_vals = m.model_instances["baseline"].out_db.series["load_" + symbol].vals
#     end_vals = m.model_instances["baseline"].out_db.series[symbol].vals
#     rel_changes = (end_vals - start_vals)/start_vals
#     large_changes = rel_changes[~(rel_changes.between(-0.0001, 0.0001))]
#     if len(large_changes) > 0:
#         print("Large deviations for symbol \n" + symbol + ":")
#         print(large_changes, "\n")
#     else:
#         print("No large deviations for symbol: " + symbol + ".\n")

In [ ]:
# from decimal import Decimal

In [ ]:
# #Demands
# print(Decimal(qD["U_EOP_tau1_1"] - mu[("U_EOP_tau1_1", "C_CO2_1")] * qS["C_CO2_1"]))
# print(Decimal(qD["EOP_tau1"] - mu.loc[("U_EOP_tau1_1", "EOP_tau1")] * qD["U_EOP_tau1_1"]))
# print(Decimal(qD["EOP_tau1_electricity"] - mu[("EOP_tau1_electricity", "EOP_tau1")] * qD["EOP_tau1"]))
# print(Decimal(qD["EOP_tau1_K"] - mu[("EOP_tau1_K", "EOP_tau1")] * qD["EOP_tau1"]))
# #zero profits
# print(Decimal(qS["C_CO2_1"]*PbT["C_CO2_1"] - qD["U_EOP_tau1_1"]*PwT["U_EOP_tau1_1"]))
# print(Decimal(qD["U_EOP_tau1_1"]*PwT["U_EOP_tau1_1"] - qD["EOP_tau1"]*PwT["EOP_tau1"]))
# print(Decimal(qD["EOP_tau1"]*PwT["EOP_tau1"] - qD["EOP_tau1_electricity"]*PwT["EOP_tau1_electricity"] - qD["EOP_tau1_K"]*PwT["EOP_tau1_K"]))